In [1]:
import os
import zipfile
import shutil
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
from PIL import Image
from keras import layers
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Concatenate, Input
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, DenseNet121
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report, f1_score, roc_curve, auc
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img

In [36]:
!pip freeze > requirements.txt

In [2]:
np.random.seed(42)

In [3]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
image_dataset_folder = 'Berrijam Datasets/' #
dataset_file = input("Enter the dataset filename: ")
if dataset_file == "Needs Respray":
    folder = zipfile.ZipFile(image_dataset_folder + "Data - Needs Respray - 2024-03-26.zip", "r")
    folder.extractall()
    folder.close()
    dataset_dir = "Data - Needs Respray - 2024-03-26"
elif dataset_file == "Is Epic Intro":
    folder = zipfile.ZipFile(image_dataset_folder + "Data - Is Epic Intro 2024-03-25.zip", "r")
    folder.extractall()
    folder.close()
    dataset_dir = "Data - Is Epic Intro 2024-03-25"
elif data_file == "Is GenAI":
    folder = zipfile.ZipFile(image_dataset_folder + "Data - Is GenAI - 2024-03-25.zip", "r")
    folder.extractall()
    folder.close()
    dataset_dir = "Data - Is GenAI - 2024-03-25"

Enter the dataset filename:  Needs Respray


In [5]:
for filename in os.listdir(dataset_dir):
    if filename.endswith('g'):
        img_path = os.path.join(dataset_dir, filename)
        img = Image.open(img_path)
        img_array = np.array(img)
        img_array = (img_array / np.max(img_array)) * 255
        img_array = img_array.astype(np.uint8)
        
        print(f"Image: {filename}")
        
        print(f"Min pixel value: {np.min(img_array)}")
        print(f"Max pixel value: {np.max(img_array)}\n")

        img = Image.fromarray(img_array)
        img.save(img_path)
        print(f"{filename} has been updated and saved.\n")

Image: paver weeds - 03.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 03.png has been updated and saved.

Image: paver weeds - 09.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 09.png has been updated and saved.

Image: paver weeds - 17.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 17.png has been updated and saved.

Image: paver weeds - 25.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 25.png has been updated and saved.

Image: paver weeds - 32.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 32.png has been updated and saved.

Image: paver weeds - 36.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 36.png has been updated and saved.

Image: paver weeds - 43.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 43.png has been updated and saved.

Image: paver weeds - 47.png
Min pixel value: 0
Max pixel value: 255

paver weeds - 47.png has been updated and saved.

Image: paver weeds - 59.png
Min pixel value: 0
M

In [6]:
for filename in os.listdir(dataset_dir):
    if filename.endswith('v'):
        csv_file = os.path.join(dataset_dir, filename)
        
Labels = pd.read_csv(csv_file)
columns = list(Labels.columns)

In [7]:
for filename in os.listdir(dataset_dir):
    if filename.endswith('v'):
        csv_file = os.path.join(dataset_dir, filename)
        
Labels = pd.read_csv(csv_file)
columns = list(Labels.columns)
label_names = Labels.iloc[:, 0]
expanded_labels = pd.get_dummies(Labels[columns[1]])
expanded_labels = expanded_labels.astype('bool')
frames = [label_names, expanded_labels]
df = pd.concat(frames, axis = 1)
folder_names = Labels[columns[1]].unique()

copy_from = f"{dataset_dir}/"
copy_to = f'Images_{dataset_dir[7:-13]}/'

if os.path.exists(f'Images_{dataset_dir[7:-13]}') == False:
    os.mkdir(f'Images_{dataset_dir[7:-13]}')
    
if os.path.exists(f'{copy_to}Train') == False:
    os.mkdir(f'{copy_to}Train')

if os.path.exists(f'{copy_to}Test') == False:
    os.mkdir(f'{copy_to}Test')

for name in folder_names:
    if os.path.exists(copy_to + 'Train/' + name) == False:
        os.mkdir(copy_to + 'Train/' + name)
    if os.path.exists(copy_to + 'Test/' + name) == False:
        os.mkdir(copy_to + 'Test/' + name)


for name in folder_names:
    files = df.Filename[df[name] == True]
    num_samples = len(files)
    num_train_samples = round(num_samples * 0.8)
    i = 0
    for f in files:
        path_from = copy_from + f
        if i < num_train_samples:
            path_to = copy_to + 'Train/' + name + '/' + f
        else:
            path_to = copy_to + 'Test/' + name + '/' + f
        shutil.copyfile(path_from, path_to)
        i += 1

In [8]:
IMAGE_HEIGHT = 900
IMAGE_WIDTH = 1200

# Define the augmentation layers
data_augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.4, 0.4),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.GaussianNoise(30)
])

def augment_and_resize(image):
    # Apply augmentations and then resize
    augmented = data_augmentation_layers(image, training=True)
    resized = tf.image.resize(augmented, [IMAGE_HEIGHT, IMAGE_WIDTH])
    return resized
    
def load_and_preprocess_image(file_path):
    # Load, decode, and preprocess the image
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_HEIGHT, IMAGE_WIDTH]) 
    image = tf.reshape(image, [IMAGE_HEIGHT, IMAGE_WIDTH, 3])
    return image

def generate_multiple_augmented_samples(image, num_samples):
    # Generate 10 augmented samples for a single image
    return tf.data.Dataset.from_tensors(image).repeat(num_samples).map(
        augment_and_resize, num_parallel_calls=tf.data.AUTOTUNE)

def augment_and_save_dataset(image_dir, save_dir, num_samples=10):
    classes = os.listdir(image_dir)
    for class_name in classes:
        class_path = os.path.join(image_dir, class_name)
        save_class_path = os.path.join(save_dir, class_name)
        os.makedirs(save_class_path, exist_ok=True)

        dataset = tf.data.Dataset.list_files(os.path.join(class_path, '*g'), shuffle=False)
        dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        
        # Use flat_map to apply the generate_multiple_augmented_samples function with num_samples
        dataset = dataset.flat_map(lambda image: generate_multiple_augmented_samples(image, num_samples))
        
        # Batch and save images
        dataset = dataset.batch(1)  # Batch each image individually if saving separately
        i = 0  # Reset counter for each class
        for batch in dataset:
            save_path = os.path.join(save_class_path, f"aug_{class_name}_{i}.png")
            tf.keras.utils.save_img(save_path, batch[0].numpy())
            i += 1

In [9]:
%%time
img_dir_train = f'Images_{dataset_dir[7:-13]}/Train/'
save_dir_train = f'Images_{dataset_dir[7:-13]}/Train/'
img_dir_test = f'Images_{dataset_dir[7:-13]}/Test/'
save_dir_test = f'Images_{dataset_dir[7:-13]}/Test/'
augment_and_save_dataset(img_dir_train, save_dir_train, num_samples = 100)
augment_and_save_dataset(img_dir_test, save_dir_test, num_samples = 1)

CPU times: total: 7min 2s
Wall time: 2min 45s


In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255)
BATCH_SIZE = 32

In [21]:
def step_decay_schedule(initial_lr, decay_factor=0.1, step_size=10):
  def schedule(epoch):
    return initial_lr * (decay_factor ** np.floor(epoch / step_size))
    
  return LearningRateScheduler(schedule)

#os.makedirs("Models", exist_ok=True)

In [11]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True 
)

In [12]:
model_checkpoint_vgg16 = ModelCheckpoint(
    filepath='Models/vgg16_best_weights.h5',  
    save_best_only=True,               
    monitor='val_loss',                
    verbose=1                          
)

In [13]:
model_checkpoint_inceptionv3 = ModelCheckpoint(
    filepath='Models/inceptionv3_best_weights.h5',  
    save_best_only=True,               
    monitor='val_loss',                
    verbose=1                          
)

In [14]:
model_checkpoint_densenet121 = ModelCheckpoint(
    filepath='Models/densenet121_best_weights.h5',  
    save_best_only=True,               
    monitor='val_loss',                
    verbose=1                          
)

In [15]:
n_epochs = 30
learning_rate = 0.001
decay_factor = 0.05
step_size = 5

In [16]:
input_shape_vgg16 = (224, 224, 3)

training_data_vgg16 = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (224, 224), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='training', shuffle = True, seed = 42)
validation_data_vgg16 = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (224, 224), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='validation', shuffle = True, seed = 42)
testing_data_vgg16 = test_datagen.flow_from_directory(directory = img_dir_test, target_size = (224, 224), color_mode = 'rgb', class_mode = 'binary', shuffle = False, seed = 42)

n_steps = training_data_vgg16.samples // BATCH_SIZE
n_val_steps = validation_data_vgg16.samples // BATCH_SIZE
optimizer = 'Adam'
reg_strength = 0.0001

Found 808 images belonging to 2 classes.
Found 202 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [19]:
#VGG16
def create_model_vgg16(input_shape, optimizer, lr, reg_strength=0.3):
    if optimizer == 'SGD':
        opt = SGD(learning_rate=lr)
    elif optimizer == 'RMSprop':
        opt = Adam(learning_rate=lr)
    elif optimizer == 'Adam':
        opt = RMSprop(learning_rate=lr)
    
    conv_base = VGG16(include_top=False,
                      weights='imagenet',
                      input_shape=input_shape)
    
    for layer in conv_base.layers:
        layer.trainable = False
        
    top_model = conv_base.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(128, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(64, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(32, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    output_layer = Dense(1, activation='sigmoid')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)
    
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [28]:
model_vgg16 = create_model_vgg16(input_shape_vgg16, optimizer, learning_rate, reg_strength=reg_strength)
lr_scheduler = step_decay_schedule(learning_rate, decay_factor, step_size)
# history_vgg16 = model_vgg16.fit(training_data_vgg16,
#                     validation_data=validation_data_vgg16,
#                     steps_per_epoch=n_steps,
#                     validation_steps=n_val_steps,
#                     epochs=n_epochs,
#                     callbacks=[lr_scheduler, early_stopping, model_checkpoint_vgg16],
#                     verbose=1)

In [29]:
print(model_vgg16)

In [23]:
img_dir_test = f'Images_{dataset_dir[7:-13]}/Test/'

classes = os.listdir(img_dir_test)
for class_name in classes:
    class_path = os.path.join(img_dir_test, class_name)
    for filename in os.listdir(class_path):
        if filename.endswith('png'):
            img_path = os.path.join(class_path, filename)
            img = tf.io.read_file(img_path)
            img = tf.image.decode_png(img, channels=3)
            img = tf.cast(img, tf.float32)
            max_val = tf.reduce_max(img)
            img_normalized = (img / max_val) * 255
        
            tf.keras.utils.save_img(img_path, img_normalized)
            print(f"{filename} has been updated and saved.\n")

aug_No_0.png has been updated and saved.

GettyImages-1023534494-e1685982394201.png has been updated and saved.

images_2.png has been updated and saved.

OOM-2023-12-26T114502.640.png has been updated and saved.

paver weeds - 59.png has been updated and saved.

Pavers-clean-Robert-Maxwell-for-FHM-JVedit.png has been updated and saved.

phb-cleanup-6.png has been updated and saved.

roof-cleaning-img.png has been updated and saved.

5-Ways-to-Prevent-Weed-Growth-Between-Paving-Stones.png has been updated and saved.

aug_Yes_0.png has been updated and saved.

download.png has been updated and saved.

download_1.png has been updated and saved.

g1YsC.png has been updated and saved.

images.png has been updated and saved.

images_1.png has been updated and saved.

paver weeds - 85.png has been updated and saved.



In [31]:
test_steps = 1
testing_data_vgg16.reset()

model_vgg16.load_weights('Models/vgg16_best_weights.h5')

predictions_vgg16 = model_vgg16.predict(testing_data_vgg16, steps=test_steps, verbose=1)

predicted_classes_vgg16 = np.where(predictions_vgg16 > 0.5, 1, 0)

true_classes = testing_data_vgg16.classes

true_classes = true_classes[:len(predicted_classes_vgg16)]

report_vgg16 = classification_report(true_classes, predicted_classes_vgg16, target_names=testing_data_vgg16.class_indices)
print(report_vgg16)

fpr_vgg16, tpr_vgg16, thresholds_vgg16 = roc_curve(true_classes, predictions_vgg16)
roc_auc_vgg16 = auc(fpr_vgg16, tpr_vgg16)

cm_vgg16 = confusion_matrix(true_classes, predicted_classes_vgg16)

1/1 [==============================] - 0s 122ms/step
              precision    recall  f1-score   support

          No       0.50      1.00      0.67         2
         Yes       0.00      0.00      0.00         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4



C:\Users\Rushi\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Rushi\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Rushi\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
predictions_vgg16

array([[0.26077202],
       [0.00131006],
       [0.29917446],
       [0.0087604 ]], dtype=float32)

In [34]:
true_classes

array([0, 0, 1, 1])

In [35]:
predicted_classes_vgg16

array([[0],
       [0],
       [0],
       [0]])

In [32]:
print(f"The F1 Score is: {f1_score(true_classes, predicted_classes_vgg16)}")

The F1 Score is: 0.0


In [ ]:
plt.figure()
lw = 2 
plt.plot(fpr_vgg16, tpr_vgg16, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc_vgg16)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
input_shape_inceptionv3 = (299, 299, 3)

training_data_inceptionv3 = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (299, 299), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='training', shuffle = True, seed = 42)
validation_data_inceptionv3 = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (299, 299), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='validation', shuffle = True, seed = 42)
testing_data_inceptionv3 = test_datagen.flow_from_directory(directory = img_dir_test, target_size = (299, 299), color_mode = 'rgb', class_mode = 'binary', shuffle = False, seed = 42)

n_steps = training_data_inceptionv3.samples // BATCH_SIZE
n_val_steps = validation_data_inceptionv3.samples // BATCH_SIZE

optimizer = 'Adam'
reg_strength = 0.0005

In [ ]:
#Inception-v3
def create_model_inceptionv3(input_shape, optimizer, lr, reg_strength=0.3):
    if optimizer == 'SGD':
        opt = SGD(learning_rate=lr)
    elif optimizer == 'RMSprop':
        opt = Adam(learning_rate=lr)
    elif optimizer == 'Adam':
        opt = RMSprop(learning_rate=lr)
    
    conv_base = InceptionV3(include_top=False,
                      weights='imagenet',
                      input_shape=input_shape)
    
    for layer in conv_base.layers:
        layer.trainable = False
        
    top_model = conv_base.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(128, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(64, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(32, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    output_layer = Dense(1, activation='sigmoid')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)
    
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model_inceptionv3 = create_model_inceptionv3(input_shape_inceptionv3, optimizer, learning_rate, reg_strength=reg_strength)
lr_scheduler = step_decay_schedule(learning_rate, decay_factor, step_size)
history_inceptionv3 = model_inceptionv3.fit(training_data_inceptionv3,
                    validation_data=validation_data_inceptionv3,
                    steps_per_epoch=n_steps,
                    validation_steps=n_val_steps,
                    epochs=n_epochs,
                    callbacks=[lr_scheduler, early_stopping, model_checkpoint_inceptionv3],
                    verbose=1)

In [ ]:
test_steps = 1
testing_data_inceptionv3.reset()

model_inceptionv3.load_weights('Models/inceptionv3_best_weights.h5')

predictions_inceptionv3 = model_inceptionv3.predict(testing_data_inceptionv3, steps=test_steps, verbose=1)

predicted_classes_inceptionv3 = np.where(predictions_inceptionv3 > 0.5, 1, 0)

true_classes = testing_data_inceptionv3.classes

true_classes = true_classes[:len(predicted_classes_inceptionv3)]

report_inceptionv3 = classification_report(true_classes, predicted_classes_inceptionv3, target_names=testing_data_inceptionv3.class_indices)
print(report_inceptionv3)

fpr_inceptionv3, tpr_inceptionv3, thresholds_inceptionv3 = roc_curve(true_classes, predictions_inceptionv3)
roc_auc_inceptionv3 = auc(fpr_inceptionv3, tpr_inceptionv3)

cm_inceptionv3 = confusion_matrix(true_classes, predicted_classes_inceptionv3)

In [ ]:
print(f"The F1 Score is: {f1_score(true_classes, predicted_classes_inceptionv3)}")

In [ ]:
plt.figure()
lw = 2 
plt.plot(fpr_inceptionv3, tpr_inceptionv3, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc_inceptionv3)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
input_shape_densenet121 = (224, 224, 3)

training_data_densenet121 = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (224, 224), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='training', shuffle = True, seed = 42)
validation_data_densenet121 = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (224, 224), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='validation', shuffle = True, seed = 42)
testing_data_densenet121 = test_datagen.flow_from_directory(directory = img_dir_test, target_size = (224, 224), color_mode = 'rgb', class_mode = 'binary', shuffle = False, seed = 42)

n_steps = training_data_densenet121.samples // BATCH_SIZE
n_val_steps = validation_data_densenet121.samples // BATCH_SIZE

optimizer = 'SGD'
reg_strength = 0.0005

In [ ]:
#DenseNet
def create_model_densenet121(input_shape, optimizer, lr, reg_strength=0.3):
    if optimizer == 'SGD':
        opt = SGD(learning_rate=lr)
    elif optimizer == 'RMSprop':
        opt = Adam(learning_rate=lr)
    elif optimizer == 'Adam':
        opt = RMSprop(learning_rate=lr)
    
    conv_base = DenseNet121(include_top=False,
                      weights='imagenet',
                      input_shape=input_shape)
    
    for layer in conv_base.layers:
        layer.trainable = False
        
    top_model = conv_base.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(128, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(64, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)    
    top_model = Dense(32, activation='relu', kernel_regularizer=l2(reg_strength))(top_model)
    top_model = BatchNormalization()(top_model)
    top_model = Dropout(0.5)(top_model)
    output_layer = Dense(1, activation='sigmoid')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)
    
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
model_densenet121 = create_model_densenet121(input_shape_densenet121, optimizer, learning_rate, reg_strength=reg_strength)
lr_scheduler = step_decay_schedule(learning_rate, decay_factor, step_size)
history_densenet121 = model_densenet121.fit(training_data_densenet121,
                    validation_data=validation_data_densenet121,
                    steps_per_epoch=n_steps,
                    validation_steps=n_val_steps,
                    epochs=n_epochs,
                    callbacks=[lr_scheduler, early_stopping, model_checkpoint_densenet121],
                    verbose=1)

In [ ]:
test_steps = 1
testing_data_densenet121.reset()

model_densenet121.load_weights('Models/densenet121_best_weights.h5')

predictions_densenet121 = model_densenet121.predict(testing_data_densenet121, steps=test_steps, verbose=1)

predicted_classes_densenet121 = np.where(predictions_densenet121 > 0.5, 1, 0)

true_classes = testing_data_densenet121.classes

true_classes = true_classes[:len(predicted_classes_densenet121)]

report_densenet121 = classification_report(true_classes, predicted_classes_densenet121, target_names=testing_data_densenet121.class_indices)
print(report_densenet121)

fpr_densenet121, tpr_densenet121, thresholds = roc_curve(true_classes, predictions_densenet121)
roc_auc_densenet121 = auc(fpr_densenet121, tpr_densenet121)

cm_densenet121 = confusion_matrix(true_classes, predicted_classes_densenet121)

In [ ]:
print(f"The F1 Score is: {f1_score(true_classes, predicted_classes_densenet121)}")

In [ ]:
plt.figure()
lw = 2 
plt.plot(fpr_densenet121, tpr_densenet121, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc_densenet121)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
training_data_ensemble = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (224, 224), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='training', shuffle = True, seed = 42)
validation_data_ensemble = train_datagen.flow_from_directory(directory = img_dir_train, target_size = (224, 224), color_mode = 'rgb', batch_size = BATCH_SIZE, class_mode = 'binary', subset='validation', shuffle = True, seed = 42)
testing_data_ensemble = test_datagen.flow_from_directory(directory = img_dir_test, target_size = (224, 224), color_mode = 'rgb', class_mode = 'binary', shuffle = False, seed = 42)

In [ ]:
model_vgg16 = load_model('Models/vgg16_best_weights.h5')
model_inception = load_model('Models/inceptionv3_best_weights.h5')
model_densenet = load_model('Models/densenet121_best_weights.h5')

for model in [model_vgg16, model_inception, model_densenet]:
    for layer in model.layers:
        layer.trainable = False

input_tensor = Input(shape=(224, 224, 3))

vgg16_features = model_vgg16(input_tensor)
inception_features = model_inception(input_tensor)
densenet_features = model_densenet(input_tensor)

concatenated_features = Concatenate()([vgg16_features, inception_features, densenet_features])

output = Dense(1, activation='sigmoid')(concatenated_features)

ensemble_model = Model(inputs=input_tensor, outputs=output)

In [ ]:
ensemble_model.compile(optimizer=Adam(learning_rate=0.0001),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])
history = ensemble_model.fit(training_data_ensemble,
                             epochs = 20,
                             validation_data = validation_data_ensemble)

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.legend()
    plt.show()

plot_history(history)


In [ ]:
min_val_loss = min(history.history['val_loss'])
min_loss_index = history.history['val_loss'].index(min_val_loss)
min_loss_acc = history.history['val_accuracy'][min_loss_index]

print("\nTraining results:")
print("\tMin val loss {:.4f} was achieved during iteration #{}".format(min_val_loss, min_loss_index + 1))
print("\tVal accuracy during min val loss is {:.4f}".format(min_loss_acc))